In [1]:
# Instalar dependências
!pip install --upgrade langchain_community
!pip install langchain langchain-groq langchain-community youtube_transcript_api pypdf
!pip install transformers sentence-transformers faiss-cpu
!pip install gradio

from huggingface_hub import notebook_login
notebook_login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Estrutura
import gradio as gr
import os
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader, YoutubeLoader, PyPDFLoader
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

In [6]:
!pip install --upgrade python-dotenv

In [10]:
import os
from dotenv import load_dotenv

# 🔹 Montar o Google Drive (se ainda não montou)
from google.colab import drive
drive.mount('/content/drive')

# 🔹 Definir o caminho do arquivo .env dentro do Drive (ajuste o caminho se necessário)
env_path = "/content/drive/MyDrive/chave.env"

# 🔹 Carregar as variáveis do arquivo .env
load_dotenv(env_path)

# 🔹 Testar se a variável foi carregada
api_key = os.getenv("GROQ_API_KEY")

if api_key:
    print("Chave carregada com sucesso!")
    os.environ["GROQ_API_KEY"] = api_key  # Define no ambiente
else:
    print("Erro: A chave não foi carregada. Verifique seu arquivo .env.")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Chave carregada com sucesso!


In [12]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

chat = ChatGroq(model='llama-3.3-70b-specdec')

In [13]:
with open(".gitignore", "w") as f:
    f.write(".env\n")


In [14]:
def resposta_bot(mensagens, documento):
    mensagem_sistema = '''
    Você é um assistente amigável chamado DataBot. Você utiliza as seguintes informações para formular as suas respostas:
    {informacoes}
    '''
    mensagens_modelo = [('system', mensagem_sistema)] # A mensagem de sistema define o comportamento do modelo.
    mensagens_modelo += mensagens
    template = ChatPromptTemplate.from_messages(mensagens_modelo)
    chain = template | chat
    return chain.invoke({'informacoes': documento}).content

In [15]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
import re



In [16]:
def carrega_site(url_site):
    loader = WebBaseLoader(url_site)
    return " ".join([doc.page_content for doc in loader.load()])

def carrega_pdf():
    loader = PyPDFLoader(caminho)
    return " ".join([doc.page_content for doc in loader.load()])

def extrair_video_id(url):
    """Extrai o video_id corretamente de diferentes formatos de URLs do YouTube"""
    padrao = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:watch\?v=|embed\/|shorts\/)|youtu\.be\/)([\w-]{11})"
    match = re.search(padrao, url)
    return match.group(1) if match else None

def carrega_youtube(url):
    try:
        # Extrai o ID do vídeo corretamente de diferentes formatos de URL
        match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11})", url)
        if not match:
            return "Erro: ID do vídeo não encontrado na URL fornecida."

        video_id = match.group(1)

        # Obtém a transcrição, preferindo legendas em português
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['pt', 'en'])

        # Junta o texto da transcrição em uma única string
        transcricao = " ".join([item['text'] for item in transcript])
        return transcricao

    except Exception as e:
        return f"Erro ao carregar a transcrição: {e}"

In [17]:
def criar_index(documento):
    if not documento.strip():
        raise ValueError("O documento fornecido esta vazio.")

    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    frases = documento.split(". ")  # Divide o texto em sentenças
    embeddings = model.encode(frases, convert_to_numpy=True)

    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    return index, model, frases

In [18]:
def buscar_documentos_relevantes(pergunta, index, model, documentos):
    pergunta_embedding = model.encode([pergunta], convert_to_numpy=True)
    _, indices = index.search(pergunta_embedding, k=3)
    return "\n".join([documentos[i] for i in indices[0]])

In [19]:
def resposta_com_rag(pergunta, index, model, documentos):
    informacoes_relevantes = buscar_documentos_relevantes(pergunta, index, model, documentos)

    if not informacoes_relevantes.strip():
        return "Não encontrei informações relevantes para responder à sua pergunta. Tente reformular ou carregar outro documento."

    mensagem_sistema = 'Você é um assistente amigável chamado DataBot. Utilize as seguintes informações para formular suas respostas:\n{informacoes}'
    mensagens_modelo = [('system', mensagem_sistema), ('user', pergunta)]

    template = ChatPromptTemplate.from_messages(mensagens_modelo)
    chain = template | chat

    return chain.invoke({'informacoes': informacoes_relevantes}).content


In [20]:
def interagir(tipo, entrada):
    global index, model, documentos
    if tipo == "Site":
        documento = carrega_site(entrada)
    elif tipo == "PDF":
        documento = carrega_pdf(entrada)
    elif tipo == "YouTube":
        documento = carrega_youtube(entrada)
    else:
        return "Opção inválida. Escolha Site, PDF ou YouTube."

    index, model, documentos = criar_index(documento)
    if not documentos:
        return "O documento carregado está vazio ou não pôde ser processado."

    return f"Documento carregado com sucesso! Agora envie uma pergunta."

def perguntar(pergunta):
    global index, model, documentos
    if index is None or model is None or not documentos:
        return "Nenhum documento carregado. Primeiro, carregue um documento antes de perguntar."
    return resposta_com_rag(pergunta, index, model, documentos)


In [21]:
# Interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 DataBot - Agente de IA")

    with gr.Row():
        opcao = gr.Radio(["Site", "PDF", "YouTube"], label="Escolha a Fonte de Dados")
        entrada = gr.Textbox(label="Digite a URL ou caminho do arquivo")
        carregar_btn = gr.Button("Carregar")

    saida_carregar = gr.Textbox(label="Status", interactive=False)
    carregar_btn.click(interagir, inputs=[opcao, entrada], outputs=saida_carregar)

    pergunta = gr.Textbox(label="Digite sua pergunta")
    perguntar_btn = gr.Button("Perguntar")
    resposta = gr.Textbox(label="Resposta do DataBot", interactive=False)

    perguntar_btn.click(perguntar, inputs=pergunta, outputs=resposta)

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cced26ba3f66784e37.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://cced26ba3f66784e37.gradio.live


In [26]:
import shutil

shutil.copy("/content/drive/MyDrive/AI_Agent_V3.ipynb", "/content/")




['Classroom', "CV's Wanderson", 'Colab Notebooks', 'Imersão ALura e Google.docx', 'Imersão ALura e Google.docx (Converted - 2024-05-10 15:51).gdoc', 'Certificados', 'Exemplo - Transição de carreira.gdoc', 'Metodista (1).gdoc', 'Metodista.gdoc', 'COCO', 'Imagens', 'CV Wanderson.Vieira.pdf', 'Documento sem título.gdoc', 'chave.env']
